# **Introduction**

* MLflow is an Open Source Machine Learning Platform. It is a better way to manage the ML lifecycle.

## **MLflow Tracking Quick Start**

**What you will learn?**
* How to `log` parameters, metrics, and a model
* The basics of the MLflow fluent API
* How to `register` a model during logging
* How to navigate to a model in the `MLflow UI`
* How to `load` a logged model for inference

#### **Step-1 - Get MLflow**

In [ ]:
# !pip install mlflow

#### **Step-2 - Start a Tracking Serve**

Reference Link - https://mlflow.org/docs/latest/getting-started/running-notebooks/index.html

In [ ]:
# Run this command in bash: mlflow server --host 127.0.0.1 --port 8080

In [ ]:
# Setting thje tracking Server URI
import mlflow

mlflow.set_tracking_uri(uri="http://<host>:<port>")

#### **Step-3 - Train a model and prepare metadata for logging**

* Loading Iris Dataset
* Training a Logistic Regression Model and evaluating it
* Preparing the model hyperparameters and calcaulating the metrics for logging

In [ ]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)

#### **Step-4 - Log the Model and its metadata to MLflow**

**The steps that we will take are:**
* Initiate an MLflow run context to start a new run that we will log the model and metadata to.
* Log model parameters and performance metrics.
* Tag the run for easy retrieval.
* Register the model in the MLflow Model Registry while logging (saving) the model.

In [ ]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

#### **Step-5 - Load the model as a python functioin and use it for inference**

**After logging the model, we can perform inference by:**

* Loading the model using MLflow’s pyfunc flavor.
* Running Predict on new data using the loaded model.

In [ ]:
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]

#### **Step-6 - View the Run in the MLflow UI**

In order to see the results of our run, we can navigate to the MLflow UI. Since we have already started the Tracking Server at http://localhost:8080, we can simply navigate to that URL in our browser.

## **Tutorial and Example**

In [1]:
#
# train_diabetes.py
#
#   MLflow model using ElasticNet (sklearn) and Plots ElasticNet Descent Paths
#
#   Uses the sklearn Diabetes dataset to predict diabetes progression using ElasticNet
#       The predicted "progression" column is a quantitative measure of disease progression one year after baseline
#       http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html
#   Combines the above with the Lasso Coordinate Descent Path Plot
#       http://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html
#       Original author: Alexandre Gramfort <alexandre.gramfort@inria.fr>; License: BSD 3 clause
#
#  Usage:
#    python train_diabetes.py 0.01 0.01
#    python train_diabetes.py 0.01 0.75
#    python train_diabetes.py 0.01 1.0
#

import sys
import warnings
from itertools import cycle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import ElasticNet, enet_path
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [2]:
# Load Diabetes datasets
diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target

# Create pandas DataFrame for sklearn ElasticNet linear_model
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = diabetes.feature_names + ["progression"]
data = pd.DataFrame(d, columns=cols)

In [3]:
# Import mlflow
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

# Import Dagshub
import dagshub

In [4]:
# Evaluate metrics
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [5]:
if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "progression" which is a quantitative measure of disease progression one year after baseline
    train_x = train.drop(["progression"], axis=1)
    test_x = test.drop(["progression"], axis=1)
    train_y = train[["progression"]]
    test_y = test[["progression"]]

    # alpha = float(sys.argv[1]) if len(sys.argv) > 1 else 0.05
    # l1_ratio = float(sys.argv[2]) if len(sys.argv) > 2 else 0.05
    alpha = 0.05
    l1_ratio = 0.05

    # Run ElasticNet
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)
    predicted_qualities = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    # Print out ElasticNet model metrics
    print(f"Elasticnet model (alpha={alpha:f}, l1_ratio={l1_ratio:f}):")
    print(f"  RMSE: {rmse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")

    # Infer model signature
    predictions = lr.predict(train_x)
    signature = infer_signature(train_x, predictions)
        
    # mlflow.set_tracking_uri(uri="http://<host>:<port>")
    remote_server_uri = "https://dagshub.com/pavanpandya/mflow-exploration"
    mlflow.set_tracking_uri(uri=remote_server_uri)

    dagshub.init("mlflow-exploration", "pavanpandya", mlflow=True)
    mlflow.start_run()

    # Log mlflow attributes for mlflow UI
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    mlflow.sklearn.log_model(lr, "model", signature=signature)

    # Compute paths
    eps = 5e-3  # the smaller it is the longer is the path

    print("Computing regularization path using the elastic net.")
    alphas_enet, coefs_enet, _ = enet_path(X, y, eps=eps, l1_ratio=l1_ratio)

    # Display results
    fig = plt.figure(1)
    ax = plt.gca()

    colors = cycle(["b", "r", "g", "c", "k"])
    neg_log_alphas_enet = -np.log10(alphas_enet)
    for coef_e, c in zip(coefs_enet, colors):
        l2 = plt.plot(neg_log_alphas_enet, coef_e, linestyle="--", c=c)

    plt.xlabel("-Log(alpha)")
    plt.ylabel("coefficients")
    title = "ElasticNet Path by alpha for l1_ratio = " + str(l1_ratio)
    plt.title(title)
    plt.axis("tight")

    # Save figures
    fig.savefig("ElasticNet-paths.png")

    # Close plot
    plt.close(fig)

    # Log artifacts (output files)
    mlflow.log_artifact("ElasticNet-paths.png")

    mlflow.end_run()

Elasticnet model (alpha=0.050000, l1_ratio=0.050000):
  RMSE: 78.59249466381223
  MAE: 66.30998032458166
  R2: 0.06607434687959957


Repository initialized!

MlflowException: API request to endpoint /api/2.0/mlflow/runs/create failed with error code 404 != 200. Response body: '<!DOCTYPE html>
<html lang="en-US">
  <head data-suburl="">
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/>
    <meta http-equiv="X-UA-Compatible" content="IE=edge"/>
    
    
      <meta name="viewport" content="width=device-width, initial-scale=1"/>
    
    <meta name="referrer" content="no-referrer"/>
    <meta name="_csrf" content="4SCI5lfeuUuyK1uNhmoxtd0npGk6MTcwNDY3MzAzNjgzNTUzODg5Mg"/>
    <meta name="_suburl" content=""/>
    <meta name="_appversion" content="9348068744edad4d1e4194a0ec852c168d325ac7" />
    <meta name="_bareRepo" content=""/>

    
    
  <script>(function(w, d, s, l, i) {
      w[l] = w[l] || [];
      w[l].push({
        'gtm.start':
          new Date().getTime(), event: 'gtm.js'
      });
      var f = d.getElementsByTagName(s)[0],
        j = d.createElement(s),
        dl = l != 'dataLayer' ? '&l=' + l : '';
  